## Derive Experiment Dataset from daily PIOMAS products to match Landy et al. (2022)

This notebook is designed to explore the daily/biweekly gridded PIOMAS SIT record.

The goals are to:
1) Calculate area, volume, total SIA, total SIV, and thickness and generate a netcdf dataset for analysis.
2) Standardize the time field to match Landy et al. (2022) Cryosat-2 product

### Prepare noteboook

In [1]:
import xarray as xr
import numpy as np
import glob
import datetime
import matplotlib.pyplot as plt
from shapely.geometry import box as BOX

### Convert PIOMAS daily data

Read in the daily SIT and SIC data.

In [2]:
sit_files = sorted(glob.glob('/glade/scratch/mollyw/external_data/SIT/PIOMAS/daily/PROCESSED/20[1-2]*.nc'))
sic_files = sorted(glob.glob('/glade/scratch/mollyw/external_data/SIC/PIOMAS/daily/PROCESSED/20[1-2]*.nc'))
DS_SIT = []
DS_SIC = []
for file1, file2 in zip(sit_files,sic_files):
    sit = xr.open_dataset(file1)
    sic = xr.open_dataset(file2)
    DS_SIT.append(sit)
    DS_SIC.append(sic)

sit_data = xr.concat(DS_SIT, dim = 't')
sit_data = sit_data.rename({'t':'time', 'day':'time'})
sic_data = xr.concat(DS_SIC, dim = 't')
sic_data = sic_data.rename({'t':'time', 'day':'time'})

Fix the time dimemsion.

In [3]:
init_time = datetime.datetime(2010,1,1)
piomas_times = [init_time + datetime.timedelta(days = x) for x in range(0,4383)]
piomas_times = [i for i in piomas_times if i not in [datetime.datetime(2012, 2, 29, 0, 0), datetime.datetime(2016, 2, 29, 0, 0), datetime.datetime(2020,2,29,0,0)]]

In [4]:
combo_ds = sit_data.copy()
combo_ds['siconc'] = sic_data.area
combos_ds['time'] = piomas_times

In [ ]:
combos_ds

### Calculate desired variables

Clean up thickness and concentration values.

In [5]:
combos_ds = combos_ds.rename({'thickness':'sithick'})
combos_ds = combos_ds.where(combos_ds.sithick > 0, np.nan)

Determine grid cell area.

In [8]:
def lat_lon_cell_area(lat_lon_grid_cell):
    """
    Calculate the area of a cell, in meters^2, on a lat/lon grid.
    
    This applies the following equation from Santinie et al. 2010.
    
    S = (λ_2 - λ_1)(sinφ_2 - sinφ_1)R^2
    
    S = surface area of cell on sphere
    λ_1, λ_2, = bands of longitude in radians
    φ_1, φ_2 = bands of latitude in radians
    R = radius of the sphere
    
    Santini, M., Taramelli, A., & Sorichetta, A. (2010). ASPHAA: A GIS‐Based 
    Algorithm to Calculate Cell Area on a Latitude‐Longitude (Geographic) 
    Regular Grid. Transactions in GIS, 14(3), 351-377.
    https://doi.org/10.1111/j.1467-9671.2010.01200.x

    Parameters
    ----------
    lat_lon_grid_cell
        A shapely box with coordinates on the lat/lon grid

    Returns
    -------
    float
        The cell area in meters^2

    """
    from numpy import radians, cos, sin
    
    # mean earth radius - https://en.wikipedia.org/wiki/Earth_radius#Mean_radius
    AVG_EARTH_RADIUS_METERS = 6371008.8
    
    west, south, east, north = lat_lon_grid_cell.bounds
    
    if abs(west - east) > 300:
        west = west + 360
        # print(east)
    
    west = radians(west)
    east = radians(east)
    south = radians(south)
    north = radians(north)
    
    return (east - west) * (sin(north) - sin(south)) * (AVG_EARTH_RADIUS_METERS**2)

In [10]:
area = np.zeros([120,360])
for x in range(0, 120-1):
    for y in range(0,360-1):
        lat1 = piomas_data.latitude[x,y]
        lat2 = piomas_data.latitude[x+1,y+1]
     
        
        lon1 = piomas_data.longitude[x,y]
        lon2 = piomas_data.longitude[x+1,y+1]
        
        box = BOX(lon1, lat1, lon2, lat2)
        
        area[x,y] = lat_lon_cell_area(box)
        
combos_ds['grid_area'] = (('x','y'),area)

Calcuate ice area, volume and corresponding pan-Arctic quantities.

In [12]:
Sea_Ice_Area = combos_ds.siconc * combos_ds.grid_area
Sea_Ice_Volume = combos_ds.sithick * Sea_Ice_Area
SIA = Sea_Ice_Area.sum(dim=['x','y'])
SIV = Sea_Ice_Volume.sum(dim=['x','y'])

In [14]:
combos_ds['siarea'] = Sea_Ice_Area
combos_ds['sivol'] = Sea_Ice_Volume
combos_ds['SIA'] = SIA
combos_ds['SIV'] = SIV

Subset the PIOMAS data to the same biweekly dates as the Landy et al. (2022) product.

In [16]:
csdata = xr.open_dataset('cs2_landy_data.nc')
time_ind = csdata.time

sub_piomas = []
for i in range(0, len(combos_ds.time)):
    if combos_ds.time.values[i] in time_ind.values:
        sub_piomas.append(combos_ds.isel(time = i))
        
biweekly = xr.concat(sub_piomas, dim = 'time')

Look at the data.

In [20]:
biweekly

<xarray.Dataset>
Dimensions:    (time: 236, x: 120, y: 360)
Coordinates:
    longitude  (x, y) float64 319.5 320.5 321.5 322.5 ... 317.9 318.4 318.8
    latitude   (x, y) float64 48.99 48.99 48.99 48.99 ... 72.71 72.71 72.71
  * time       (time) datetime64[ns] 2010-10-09 2010-10-24 ... 2020-07-24
Dimensions without coordinates: x, y
Data variables:
    sithick    (time, x, y) float64 nan nan nan nan nan ... nan nan nan nan nan
    siconc     (time, x, y) float64 nan nan nan nan nan ... nan nan nan nan nan
    grid_area  (time, x, y) float64 4.839e+09 4.839e+09 4.839e+09 ... 0.0 0.0
    siarea     (time, x, y) float64 nan nan nan nan nan ... nan nan nan nan nan
    sivol      (time, x, y) float64 nan nan nan nan nan ... nan nan nan nan nan
    SIA        (time) float64 3.539e+12 4.811e+12 ... 4.096e+12 3.081e+12
    SIV        (time) float64 3.734e+12 4.793e+12 ... 6.092e+12 4.328e+12
Attributes:
    data_name:      Daily mean Piomas sea ice thickness data
    description:    Sea ice thickness in meters on the native 360x120 grid, \...
    year:           These data are for the year 2010
    citation:       When using this data please use the citation: \n         ...
    code to read:     # Example code to read a month of this data \n    \n   ...
    python author:  Robbie Mallett wrote this python code. If there's a probl...

Write the data to new netcdf files.

In [23]:
biweekly.to_netcdf('piomas_biweekly_data.nc')

In [26]:
combos_ds.to_netcdf('piomas_daily_data.nc')